## Modeling2-1_Logistic-Regression
* 기상청 날씨와 주어진 공장내부정보로  결로 예측
    * 우선은 모델링을 위해 24시간 후 아닌, 주어진 시점에서의 날씨와 공장정보로 결로예측해보기.
* 결측값 처리
    * 선형보간법으로 기상청 날씨 결측값 채우기

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
from itertools import cycle
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [ ]:
# jupyter notebook cell 너비 조절
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 한글깨짐 문제
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
plant1_train=pd.read_csv('./data/plant1_train.csv',index_col=[0],parse_dates=['plant1_train.mea_ddhr'])
plant2_train=pd.read_csv('./data/plant2_train.csv',index_col=[0],parse_dates=['plant2_train.mea_ddhr'])
plant_test=pd.read_csv('./data/plant_test.csv',index_col=[0],parse_dates=['plant_test.mea_ddhr'])

# 서산
seosan=pd.read_csv('./data/asos_서산_시간별.csv',index_col=None,parse_dates=['일시'])
seosan=seosan.rename(columns={'일시':'datetime','기온(°C)':'tem','습도(%)':'hum','이슬점온도(°C)':'dew','증기압(hPa)':'vapor'})
seosan=seosan[['datetime','tem','hum','vapor','dew']]

# test데이터셋과 동일하게 train 데이터 전처리
plant_train=pd.read_csv('./data/plant_train.csv',index_col=[0],parse_dates=['datetime'])

In [ ]:
# 공장 컬럼명 정리
plant = [plant1_train, plant2_train, plant_test]

for p in plant:
    p.columns = [col.split('.')[1] for col in p.columns]    
    p.rename(columns={'mea_ddhr':'datetime'}, inplace=True)    

#### 공장별 loc별 데이터 가져오기

In [ ]:
# 컬럼에 loc1이 있는 값과 datetime만 가져오기.
### plant1_train데이터 사용시.
loc_col=[]
for c in plant1_train.columns:
    if 'loc1' in c or 'datetime' in c:
        loc_col.append(c)
print(loc_col)

#### plant_train 데이터의 datetime기준으로 기상청날씨 left join
* cond가 nan인 row는 모두 제거
* 나머지 결측치는 시계열 날짜 index를 기준으로 결측값 보간

In [ ]:
# datetime컬럼을 키값으로, plant_train데이터기준으로 left join
df_merge=pd.merge(plant_train,seosan,on='datetime',how='left')

# 결로(cond)가 결측치인 row 모두 제거.
df_merge=df_merge.iloc[df_merge['cond'].dropna(axis=0).index]

# 기상청 날씨의 결측치는 선형보간법으로 채우기.
df_merge=df_merge.interpolate()
df_merge

#### plant별 loc별 데이터 가져오기

In [ ]:
def select_plant_loc(data, plant, loc):
    result=data.loc[(data['plant']==plant) & (data['loc']==loc)]
    return result